In [1]:
import sys
from collections import namedtuple
import numpy as np
from uncertainties import ufloat, unumpy
import uncertainties
import scipy
import matplotlib
import matplotlib.pyplot as plt
import mplhep as hep
plt.style.use(hep.style.CMS)
import awkward as ak
import pandas as pd
import uproot

sys.path.append("..")
from HistogramLib.store import HistogramStore
from HistogramLib.plot_manager import PlotManager
from HistogramLib.bokeh.histogram_widget import *
from hists.parameters import beamEnergies, synchrotronBeamEnergiesMap
from hists.store import HistogramId
from bokeh_apps.widgets import *

# From histograms

In [2]:
hist_folder = '/grid_mnt/data_cms_upgrade/cuisset/testbeam18/clue3d/v33'
histStore = HistogramStore(hist_folder, HistogramId)

In [3]:
h1 = histStore.get(HistogramId("RechitsTotalEnergyPerEvent", "cmssw", "data")).getHistogram(HistogramKind.COUNT)
h1[{"beamEnergy":hist.loc(20), "rechits_energy_sum":slice(hist.loc(15), hist.loc(30))}]

Hist(Regular(86, 14.875, 29.925, name='rechits_energy_sum', label='Total reconstructed energy per event (GeV)'), storage=Double()) # Sum: 19432.0 (19969.0 with flow)

# From CLUE_clusters.root

In [4]:
tree = uproot.open('/data_cms_upgrade/cuisset/testbeam18/clue3d/v31/cmssw/data/CLUE_clusters.root:clusters')
arrays = tree.arrays(entry_stop=100)

In [5]:
ak.max(arrays.clus3D_energy, axis=-1)

<Array [144, 289, 282, 282, 282, ..., 285, 280, 284, 281] type='100 * ?float32'>

In [6]:
def getTotalEnergyRechits(events:ak.Array):
    return ak.sum(events.rechits_energy, axis=-1)
def getTotalEnergyCLUE(events:ak.Array):
    return ak.sum(events.clus2D_energy, axis=-1)
def getTotalEnergyCLUE3D(events:ak.Array, mainTracksterOnly=True):
    if mainTracksterOnly:
        return ak.max(events.clus3D_energy, axis=-1)
    else:
        return ak.sum(events.clus3D_energy, axis=-1)

In [7]:
def filterFromDf(events:ak.Array, df:pd.DataFrame):
    cut = np.isin(events.beamEnergy, df.beamEnergy) & np.isin(events.event, df.event) & np.isin(events.ntupleNumber, df.ntupleNumber)
    return events[~cut] 